# Sells for Via Uno



Objective:

The goal of this notebook is to construct and preprocess the "Via Uno" database for our sales analysis project. This involves extracting, cleaning, and integrating data from various sources to ensure a consistent and reliable dataset. By establishing a robust database, we lay the groundwork for further exploratory analysis, predictive modeling, and optimization strategies to enhance sales performance.


First we import the necessary librarys

In [1]:
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)  # Para mostrar todas las filas
pd.set_option('display.max_columns', None)  # Para mostrar todas las columnas
pd.set_option('display.width', None)  # Para que no haya corte de texto
pd.set_option('display.max_colwidth', None)  # Para que no haya corte en el contenido de las celdas


# Extract of data

We import the necesary datas 

In [2]:
principal = pd.read_excel('/home/sebastian/Documents/Portfolio/Ventas via uno/inf vtas sobre costos 2024.xlsx')
cost= pd.read_excel('/home/sebastian/Documents/Portfolio/Ventas via uno/costos 26-02-25.xlsx')
year= pd.read_excel('/home/sebastian/Documents/Portfolio/Ventas via uno/BASE VENTA 2024 2025.xlsx',skiprows=11)



In [3]:
def apply_negative_abs(df, filter_column, filter_value, target_column):
    """
    Converts values in the target_column to negative if they match the filter_value in filter_column.

    Parameters:
    df (pd.DataFrame): Input DataFrame.
    filter_column (str): Column name to filter by.
    filter_value (str): Value in filter_column that triggers the transformation.
    target_column (str): Column where abs() * -1 will be applied.

    Returns:
    pd.DataFrame: DataFrame with updated values.
    """
    df.loc[df[filter_column] == filter_value, target_column] = df.loc[df[filter_column] == filter_value, target_column].abs() * -1
    return df

# Data cleaning

#### First clean data of cost

In [4]:
cost.head(5)

SKU                  Variante  Costo Neto
0  22231604002005  Sandalia Azul Via Uno 35       10969
1  22231604002006  Sandalia Azul Via Uno 36       10969
2  22231604002007  Sandalia Azul Via Uno 37       10969
3  22231604002008  Sandalia Azul Via Uno 38       10969
4  22231604002009  Sandalia Azul Via Uno 39       10969

In [5]:
cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8025 entries, 0 to 8024
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SKU         8025 non-null   object
 1   Variante    8025 non-null   object
 2   Costo Neto  8025 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 188.2+ KB


In [6]:
cost['SKU']=cost['SKU'].str.replace('"', '', regex=False)
cost['SKU']=cost['SKU'].str.replace("=", '', regex=False)
cost.head()

SKU                  Variante  Costo Neto
0  22231604002005  Sandalia Azul Via Uno 35       10969
1  22231604002006  Sandalia Azul Via Uno 36       10969
2  22231604002007  Sandalia Azul Via Uno 37       10969
3  22231604002008  Sandalia Azul Via Uno 38       10969
4  22231604002009  Sandalia Azul Via Uno 39       10969

Create an diccionary with relation between SKU and Costo Neto

In [7]:
#Create a dictionary with the cost of each SKU
cost_dict = dict(zip(cost['SKU'], cost['Costo Neto']))
cost_dict

#Count per SKU
count = cost.groupby('SKU').size().reset_index(name='Cantidad')
count_dict = count.set_index('SKU')['Cantidad'].to_dict()

#### Cleaning year

In [8]:
year.head(5)

Tipo Documento Nº Documento           Tracking Number  \
0  BOLETA ELECTRÓNICA T        39257  679d8d919190c8aa779c2102   
1  BOLETA ELECTRÓNICA T        39256  679d81ebb99ccf26edeaa5f5   
2  BOLETA ELECTRÓNICA T        39255  679d80e6b99ccf26edeaa542   
3  BOLETA ELECTRÓNICA T        39254  679d716c691a0b5ffe9a9bea   
4  BOLETA ELECTRÓNICA T        39253  679d61189190c8aa779bf7cf   

                     Cliente Rut Cliente Condicion de Venta  \
0     natalia vergarag gaete  17635915-3             Al día   
1              Nathaly Muñoz  16545211-9             Al día   
2  Sthefany Vilches Espinoza  17874155-1             Al día   
3    Paulina Zarate Gonzalez  13680930-K             Al día   
4             Marjorie Bravo  16297286-3             Al día   

         Fecha Emisión    Fecha Vencimiento Código Tributario  \
0  2025-01-31 00:00:00  2025-01-31 00:00:00                39   
1  2025-01-31 00:00:00  2025-01-31 00:00:00                39   
2  2025-01-31 00:00:00  2025-01-31 00:00:00                39   
3  2025-01-31 00:00:00  2025-01-31 00:00:00                39   
4  2025-01-31 00:00:00  2025-01-31 00:00:00                39   

   Fecha de generacion Fecha de declaracion Informado SII  \
0  2025-02-01 00:00:00  2025-01-31 00:00:00     Informado   
1  2025-02-01 00:00:00  2025-01-31 00:00:00     Informado   
2  2025-02-01 00:00:00  2025-01-31 00:00:00     Informado   
3  2025-02-01 00:00:00  2025-01-31 00:00:00     Informado   
4  2025-01-31 00:00:00  2025-01-31 00:00:00     Informado   

  Monto Neto Documento Monto Impuestos Documento Monto Exento Documento  \
0                14277                      2713                      0   
1                18374                      3491                      0   
2                23521                      4469                      0   
3                29403                      5587                      0   
4                50997                      9690                      0   

  Monto Documento Monto Adeudado ABONO DE CLIENTES CHEQUE CREDITO EFECTIVO  \
0           16990              0                 0      0       0        0   
1           21865              0                 0      0       0        0   
2           27990              0                 0      0       0        0   
3           34990              0                 0      0       0        0   
4           60687              0                 0      0       0        0   

  FACTORING Mercado Pago NOTA CREDITO DEVOLUCION TARJETA CREDITO  \
0         0            0                       0               0   
1         0            0                       0               0   
2         0            0                       0               0   
3         0            0                       0               0   
4         0            0                       0               0   

  TARJETA DEBITO TRANSFERENCIA BANCARIA WEBPAY Numero Comprobante Integrado  \
0              0                  16990      0                          NaN   
1              0                  21865      0                          NaN   
2              0                  27990      0                          NaN   
3              0                  34990      0                          NaN   
4              0                  60687      0                          NaN   

  Nº Comprobante Banco Número BancoValue FACTURA COMPRA FACTORING.1  \
0            NaN   NaN    NaN        NaN            NaN         NaN   
1            NaN   NaN    NaN        NaN            NaN         NaN   
2            NaN   NaN    NaN        NaN            NaN         NaN   
3            NaN   NaN    NaN        NaN            NaN         NaN   
4            NaN   NaN    NaN        NaN            NaN         NaN   

                 Emisor      Vendedor     Sucursal   Estado Orden de compra  \
0  San Mateo Spa ONLINE  VENTA ONLINE  Casa Matriz  Vigente             NaN   
1  San Mateo Spa ONLINE  VENTA ONLINE  Casa Matriz  Vigente             NaN   
2  San 

In [9]:
year.tail(5)

Tipo Documento  Nº Documento  Tracking Number  \
19524  NOTA DE DÉBITO ELECTRÓNICA (3)           NaN              NaN   
19525                  Tipo Documento  Nº Documento  Tracking Number   
19526      NOTA DE DÉBITO ELECTRÓNICA            10              NaN   
19527      NOTA DE DÉBITO ELECTRÓNICA             9              NaN   
19528      NOTA DE DÉBITO ELECTRÓNICA             8              NaN   

                                   Cliente  Rut Cliente   Condicion de Venta  \
19524                                  NaN          NaN                  NaN   
19525                              Cliente  Rut Cliente        Fecha Emisión   
19526  CARCUR HERMANOS Y COMPANIA LIMITADA   80161100-1  2024-12-27 00:00:00   
19527                        Yuselys Munoz   26567490-9  2024-04-30 00:00:00   
19528                       Susana Triviño   18974141-3  2024-04-30 00:00:00   

             Fecha Emisión  Fecha Vencimiento    Código Tributario  \
19524                  NaN                NaN                  NaN   
19525    Fecha Vencimiento  Código Tributario  Fecha de generacion   
19526  2024-12-27 00:00:00                 56  2024-12-27 00:00:00   
19527  2024-04-30 00:00:00                 56  2024-05-13 00:00:00   
19528  2024-04-30 00:00:00                 56  2024-05-13 00:00:00   

        Fecha de generacion Fecha de declaracion         Informado SII  \
19524                   NaN                  NaN                   NaN   
19525  Fecha de declaracion        Informado SII  Monto Neto Documento   
19526                   NaN         Por Informar                179358   
19527   2024-05-13 00:00:00            Informado                 12597   
19528   2024-05-13 00:00:00            Informado                  5874   

            Monto Neto Documento Monto Impuestos Documento  \
19524                        NaN                       NaN   
19525  Monto Impuestos Documento    Monto Exento Documento   
19526                      34078                         0   
19527                       2393                         0   
19528                       1116                         0   

      Monto Exento Documento    Monto Documento Monto Adeudado  \
19524                    NaN                NaN            NaN   
19525        Monto Documento  ABONO DE CLIENTES         CHEQUE   
19526                 213436             119923              0   
19527                  14990                  0              0   
19528                   6990                  0              0   

      ABONO DE CLIENTES    CHEQUE    CREDITO      EFECTIVO  \
19524               NaN       NaN        NaN           NaN   
19525           CREDITO  EFECTIVO  FACTORING  Mercado Pago   
19526            213436         0          0             0   
19527                 0     14990          0             0   
19528                 0         0          0             0   

                     FACTORING     Mercado Pago NOTA CREDITO DEVOLUCION  \
19524                      NaN              NaN                     NaN   
19525  NOTA CREDITO DEVOLUCION  TARJETA CREDITO          TARJETA DEBITO   
19526                        0                0                       0   
19527                        0                0                       0   
19528                        0                0                       0   

              TARJETA CREDITO TARJETA DEBITO        TRANSFERENCIA BANCARIA  \
19524                     NaN            NaN                           NaN   
19525  TRANSFERENCIA BANCARIA         WEBPAY  Numero Comprobante Integrado   
19526                       0              0                           NaN   
19527                       0              0                           NaN   
19528                       0              0                           NaN   

               WEBPAY Numero Comprobante Integrado Nº Comprobante       Banco  \
19524             NaN                          NaN            NaN         NaN   
19525  Nº Comproba

The datasets has multi tables.

In [10]:
Repeats_headers = ['Tipo Documento', 'Nº Documento', 'Rut Cliente']

# Identificar las filas donde aparecen los encabezados
header_indices = year[year['Tipo Documento'].isin(Repeats_headers)].index.tolist()

# Agregar la última fila como un límite para capturar la última tabla
header_indices.append(len(year))

# Lista donde guardaremos las tablas separadas
tables = []

# Verificar si hay una tabla antes del primer encabezado
if header_indices[0] > 0:
    first_table = year.iloc[:header_indices[0]].reset_index(drop=True)
    if not first_table.empty:
        tables.append(first_table)

# Separar las tablas detectadas y asignar los encabezados correctos
for i in range(len(header_indices) - 1):
    start = header_indices[i]  # Fila donde está el encabezado
    end = header_indices[i + 1]  # Hasta el siguiente encabezado

    # Extraer los datos y resetear el índice
    table = year.iloc[start:end].reset_index(drop=True)
    
    # Asignar la primera fila como encabezado
    table.columns = table.iloc[0]  # Convertir la primera fila en nombres de columnas
    table = table[1:].reset_index(drop=True)  # Eliminar la fila de encabezado original
    if i < 4:
        table=table.iloc[:-2]



    if not table.empty:
        tables.append(table)

columns_fill=["Documentos Relacionados (Tipo Documento / Nº Documento / Fecha Emisión / Monto del documento )",'Marketplace']


In [11]:
# Arrangements for Documentos Relacionados (Tipo Documento / Nº Documento / Fecha Emisión / Monto del documento )
for table in tables:
    table['Nº Documento'] =table['Nº Documento'].astype(str).str.strip()
    table[columns_fill] = table[columns_fill].fillna("Sin datos")
    table['Nº Documento new'] = table['Documentos Relacionados (Tipo Documento / Nº Documento / Fecha Emisión / Monto del documento )'].str.split('/').str[1]
    table['Nº Documento'] = table['Nº Documento'].str.replace('"', '').str.replace("'", "")


marketplace_dict=tables[0].set_index('Nº Documento')['Marketplace'].to_dict()

# search Nº Documento new (Nº Documento on dictionary) in the dictionary and take the value of the marketplace
tables[4]['Nº Documento new']=tables[4]['Nº Documento new'].astype(str).str.strip()
tables[4]['Marketplace'] = tables[4]['Nº Documento new'].map(marketplace_dict).fillna("Sin datos")
marketplace_dict.update(tables[4].set_index('Nº Documento')['Marketplace'].to_dict()) #update the dictionary with the new values
tables[4].head()

0               Tipo Documento Nº Documento Tracking Number  \
0  NOTA DE CRÉDITO ELECTRÓNICA         6827             NaN   
1  NOTA DE CRÉDITO ELECTRÓNICA         6826             NaN   
2  NOTA DE CRÉDITO ELECTRÓNICA         6825             NaN   
3  NOTA DE CRÉDITO ELECTRÓNICA         6824             NaN   
4  NOTA DE CRÉDITO ELECTRÓNICA         6822             NaN   

0                       Cliente Rut Cliente        Fecha Emisión  \
0       Marie Helene Ducheylard  10251278-2  2025-01-31 00:00:00   
1  Pamela Del Carmen Pena Munoz  15393062-7  2025-01-31 00:00:00   
2              Cecilia Riquelme   9748952-1  2025-01-31 00:00:00   
3                Jeannette Vera  10051231-9  2025-01-31 00:00:00   
4                    Karim Lazo  12465332-0  2025-01-31 00:00:00   

0    Fecha Vencimiento Código Tributario  Fecha de generacion  \
0  2025-01-31 00:00:00                61  2025-02-10 00:00:00   
1  2025-01-31 00:00:00                61  2025-02-10 00:00:00   
2  2025-01-31 00:00:00                61  2025-02-10 00:00:00   
3  2025-01-31 00:00:00                61  2025-02-10 00:00:00   
4  2025-01-31 00:00:00                61  2025-02-08 00:00:00   

0 Fecha de declaracion Informado SII Monto Neto Documento  \
0  2025-02-10 00:00:00     Informado                20992   
1  2025-02-10 00:00:00     Informado                34445   
2  2025-02-10 00:00:00     Informado                19319   
3  2025-02-10 00:00:00     Informado                32756   
4  2025-02-08 00:00:00     Informado                17639   

0 Monto Impuestos Documento Monto Exento Documento Monto Documento  \
0                      3988                      0           24980   
1                      6545                      0           40990   
2                      3671                      0           22990   
3                      6224                      0           38980   
4                      3351                      0           20990   

0                Emisor      Vendedor     Sucursal   Estado Orden de compra  \
0  San Mateo Spa ONLINE  VENTA ONLINE  Casa Matriz  Vigente             NaN   
1  San Mateo Spa ONLINE  VENTA ONLINE  Casa Matriz  Vigente             NaN   
2  San Mateo Spa ONLINE  VENTA ONLINE  Casa Matriz  Vigente             NaN   
3  San Mateo Spa ONLINE  VENTA ONLINE  Casa Matriz  Vigente             NaN   
4  San Mateo Spa ONLINE  VENTA ONLINE  Casa Matriz  Vigente             NaN   

0 Nota Obser. Nombre Contacto Fono Contacto Peso Neto Peso Bruto Volumen  \
0  NaN    NaN             NaN           NaN       NaN        NaN     NaN   
1  NaN    NaN             NaN           NaN       NaN        NaN     NaN   
2  NaN    NaN             NaN           NaN       NaN        NaN     NaN   
3  NaN    NaN             NaN           NaN       NaN        NaN     NaN   
4  NaN    NaN             NaN           NaN       NaN        NaN     NaN   

0 PERIODO DEL SERVICIO Marketplace Nro. Orden Dirección Ciudad Comuna  \
0                  NaN      ripley        NaN      STGO   STGO   STGO   
1                  NaN      ripley        NaN      STGO   STGO   STGO   
2                  NaN      ripley        NaN      STGO   STGO   STGO   
3                  NaN      ripley        NaN      STGO   STGO   STGO   
4                  NaN      ripley        NaN      STGO   STGO   STGO   

0 Documentos Relacionados (Tipo Documento / Nº Documento / Fecha Emisión / Monto del documento )  \
0                                               [BOLETA ELECTRÓNICA T/ 38978/ 2025-01-26/ 24980]   
1                                               [BOLETA ELECTRÓNICA T/ 38902/ 2025-01-24/ 41980]   
2                                               [BOLETA ELECTRÓNICA T/ 38594/ 2025-01-17/ 23980]   
3                                               [BOLETA ELECTRÓNICA T/ 38369/ 2025-01-13/ 38980]   
4                                               [BOLETA ELECTRÓNICA T/ 38175/ 2025-01-09/ 20990]   

0 Documentos de Referencia Correo Cliente  NaN  NaN  NaN  NaN

In [12]:
# Now for the other tables
tables[5]['Nº Documento new']=tables[5]['Nº Documento new'].astype(str).str.strip()
tables[5]['Marketplace'] = tables[5]['Nº Documento new'].map(marketplace_dict).fillna("Sin datos")
marketplace_dict.update(tables[5].set_index('Nº Documento')['Marketplace'].to_dict())

tables[1]['Nº Documento new']=tables[1]['Nº Documento new'].astype(str).str.strip()
tables[1]['Marketplace'] = tables[1]['Nº Documento new'].map(marketplace_dict).fillna("Sin datos")
marketplace_dict.update(tables[1].set_index('Nº Documento')['Marketplace'].to_dict())


tables[2]['Nº Documento new']=tables[2]['Nº Documento new'].astype(str).str.strip()
tables[2]['Marketplace'] = tables[2]['Nº Documento new'].map(marketplace_dict).fillna("Sin datos")
marketplace_dict.update(tables[2].set_index('Nº Documento')['Marketplace'].to_dict())


tables[3]['Nº Documento new']=tables[3]['Nº Documento new'].astype(str).str.strip()
tables[3]['Marketplace'] = tables[3]['Nº Documento new'].map(marketplace_dict).fillna("Sin datos")
marketplace_dict.update(tables[3].set_index('Nº Documento')['Marketplace'].to_dict())
tables[3].head()
principal.head(5)

Tipo Movimiento        Tipo Documento  Número Documento Fecha de Emisión  \
0           venta  BOLETA ELECTRÓNICA T           21923.0       01/01/2024   
1           venta  BOLETA ELECTRÓNICA T           21924.0       01/01/2024   
2           venta  BOLETA ELECTRÓNICA T           21925.0       01/01/2024   
3           venta  BOLETA ELECTRÓNICA T           21926.0       01/01/2024   
4           venta  BOLETA ELECTRÓNICA T           21927.0       01/01/2024   

            Tracking number   Fecha y Hora Venta     Sucursal      Vendedor  \
0  659208472266bd053b66938c  31/12/2023 21:33:10  Casa Matriz  VENTA ONLINE   
1  659213862266bd053e66957b  31/12/2023 22:21:10  Casa Matriz  VENTA ONLINE   
2  659239f57e379f0505d14b6e  01/01/2024 01:05:09  Casa Matriz  VENTA ONLINE   
3  659239f9cb05a8054ae57022  01/01/2024 01:05:13  Casa Matriz  VENTA ONLINE   
4  65923dd57e379f0505d14ba0  01/01/2024 01:21:41  Casa Matriz  VENTA ONLINE   

     Cliente Nombre Cliente RUT              Cliente Email Cliente Dirección  \
0   viviana salinas  13617340-5                        NaN               NaN   
1     Silvia Psijas   6693321-0                        NaN               NaN   
2  Priscilla Bustos  16753486-4                        NaN               NaN   
3  Priscilla Bustos  16753486-4                        NaN               NaN   
4  Carolina Montiel  15663772-6  montieliglesias@gmail.com           Caren     

              Cliente Comuna Cliente Ciudad        Lista de precio  \
0  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   
1  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   
2  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   
3  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   
4  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   

    Tipo de entrega Moneda Tipo de Producto/Servicio           SKU  \
0  retiro en tienda    CLP                     Botin  1.223141e+13   
1  retiro en tienda    CLP                     Botin  1.223141e+13   
2  retiro en tienda    CLP                  Sandalia  2.232361e+13   
3  retiro en tienda    CLP                  Sandalia  2.232361e+13   
4  retiro en tienda    CLP                      Bota  1.223130e+13   

                 Producto/Servicio  Variante  Otros Atributos  Marca  \
0    Via Uno Botin 12231407 Marron  37 Negro              NaN    NaN   
1    Via Uno Botin 12231407 Marron        37              NaN    NaN   
2  Via Uno Sandalia 22323605 Negro        36              NaN    NaN   
3  Via Uno Sandalia 22323605 Negro        36              NaN    NaN   
4      Via Uno Bota 12231303 Negro        38              NaN    NaN   

   Detalle de Productos/Servicios Pack/Promo  Precio base  \
0                                        NaN      10076.0   
1                                        NaN      10076.0   
2                                        NaN      12597.0   
3                                        NaN      12597.0   
4                                        NaN      10916.0   

   Precio Neto Unitario Precio Bruto Unitario  Cantidad  Subtotal Neto  \
0               10076.0                 11990         1          10076   
1               10076.0                 11990         1          10076   
2               12597.0                 14990         1          12597   
3               12597.0                 14990         1          12597   
4               10916.0                 12990         1          10916   

   Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
0                1914           11990            NaN               0   
1                1914           11990            NaN               0   
2                2393           14990            NaN               0   
3                2393           14990            NaN               0   
4                2074           12990            NaN               0   

   Descuento Bruto  % Descuento  Costo neto unitario  Cos

In [13]:
tables[0].head(5)

Tipo Documento Nº Documento           Tracking Number  \
0  BOLETA ELECTRÓNICA T        39257  679d8d919190c8aa779c2102   
1  BOLETA ELECTRÓNICA T        39256  679d81ebb99ccf26edeaa5f5   
2  BOLETA ELECTRÓNICA T        39255  679d80e6b99ccf26edeaa542   
3  BOLETA ELECTRÓNICA T        39254  679d716c691a0b5ffe9a9bea   
4  BOLETA ELECTRÓNICA T        39253  679d61189190c8aa779bf7cf   

                     Cliente Rut Cliente Condicion de Venta  \
0     natalia vergarag gaete  17635915-3             Al día   
1              Nathaly Muñoz  16545211-9             Al día   
2  Sthefany Vilches Espinoza  17874155-1             Al día   
3    Paulina Zarate Gonzalez  13680930-K             Al día   
4             Marjorie Bravo  16297286-3             Al día   

         Fecha Emisión    Fecha Vencimiento Código Tributario  \
0  2025-01-31 00:00:00  2025-01-31 00:00:00                39   
1  2025-01-31 00:00:00  2025-01-31 00:00:00                39   
2  2025-01-31 00:00:00  2025-01-31 00:00:00                39   
3  2025-01-31 00:00:00  2025-01-31 00:00:00                39   
4  2025-01-31 00:00:00  2025-01-31 00:00:00                39   

   Fecha de generacion Fecha de declaracion Informado SII  \
0  2025-02-01 00:00:00  2025-01-31 00:00:00     Informado   
1  2025-02-01 00:00:00  2025-01-31 00:00:00     Informado   
2  2025-02-01 00:00:00  2025-01-31 00:00:00     Informado   
3  2025-02-01 00:00:00  2025-01-31 00:00:00     Informado   
4  2025-01-31 00:00:00  2025-01-31 00:00:00     Informado   

  Monto Neto Documento Monto Impuestos Documento Monto Exento Documento  \
0                14277                      2713                      0   
1                18374                      3491                      0   
2                23521                      4469                      0   
3                29403                      5587                      0   
4                50997                      9690                      0   

  Monto Documento Monto Adeudado ABONO DE CLIENTES CHEQUE CREDITO EFECTIVO  \
0           16990              0                 0      0       0        0   
1           21865              0                 0      0       0        0   
2           27990              0                 0      0       0        0   
3           34990              0                 0      0       0        0   
4           60687              0                 0      0       0        0   

  FACTORING Mercado Pago NOTA CREDITO DEVOLUCION TARJETA CREDITO  \
0         0            0                       0               0   
1         0            0                       0               0   
2         0            0                       0               0   
3         0            0                       0               0   
4         0            0                       0               0   

  TARJETA DEBITO TRANSFERENCIA BANCARIA WEBPAY Numero Comprobante Integrado  \
0              0                  16990      0                          NaN   
1              0                  21865      0                          NaN   
2              0                  27990      0                          NaN   
3              0                  34990      0                          NaN   
4              0                  60687      0                          NaN   

  Nº Comprobante Banco Número BancoValue FACTURA COMPRA FACTORING.1  \
0            NaN   NaN    NaN        NaN            NaN         NaN   
1            NaN   NaN    NaN        NaN            NaN         NaN   
2            NaN   NaN    NaN        NaN            NaN         NaN   
3            NaN   NaN    NaN        NaN            NaN         NaN   
4            NaN   NaN    NaN        NaN            NaN         NaN   

                 Emisor      Vendedor     Sucursal   Estado Orden de compra  \
0  San Mateo Spa ONLINE  VENTA ONLINE  Casa Matriz  Vigente             NaN   
1  San Mateo Spa ONLINE  VENTA ONLINE  Casa Matriz  Vigente             NaN   
2  San 

In [14]:

# Repeat but with the dictionary updated
for table in tables:
    table['Nº Documento'] =table['Nº Documento'].astype(str).str.strip()
    table['Nº Documento new']=table['Nº Documento new'].astype(str).str.strip()
    table.loc[table['Marketplace'] == "Sin datos", 'Marketplace'] = table.loc[table['Marketplace'] == "Sin datos", 'Nº Documento new'].map(marketplace_dict).fillna("Sin datos")

# Concatenate all the tables
for i in range(len(tables)):
    # Eliminar columnas duplicadas dejando solo la primera ocurrencia
    tables[i] = tables[i].loc[:, ~tables[i].columns.duplicated()]
tables = [table.reset_index(drop=True) for table in tables]
final_table = pd.concat(tables, ignore_index=True, sort=False)
final_table= final_table.loc[:, ~pd.isna(final_table.columns)]
final_table.tail(5)

Tipo Documento Nº Documento Tracking Number  \
19511  NOTA DE CRÉDITO ELECTRÓNICA         3427             NaN   
19512  NOTA DE CRÉDITO ELECTRÓNICA         3421             NaN   
19513   NOTA DE DÉBITO ELECTRÓNICA           10             NaN   
19514   NOTA DE DÉBITO ELECTRÓNICA            9             NaN   
19515   NOTA DE DÉBITO ELECTRÓNICA            8             NaN   

                                   Cliente Rut Cliente Condicion de Venta  \
19511                    victoria carrasco  11844669-0                NaN   
19512                        jezabel mella  17395619-3                NaN   
19513  CARCUR HERMANOS Y COMPANIA LIMITADA  80161100-1                NaN   
19514                        Yuselys Munoz  26567490-9                NaN   
19515                       Susana Triviño  18974141-3                NaN   

             Fecha Emisión    Fecha Vencimiento Código Tributario  \
19511  2024-02-01 00:00:00  2024-02-01 00:00:00                61   
19512  2024-02-01 00:00:00  2024-02-01 00:00:00                61   
19513  2024-12-27 00:00:00  2024-12-27 00:00:00                56   
19514  2024-04-30 00:00:00  2024-04-30 00:00:00                56   
19515  2024-04-30 00:00:00  2024-04-30 00:00:00                56   

       Fecha de generacion Fecha de declaracion Informado SII  \
19511  2024-02-01 00:00:00  2024-02-01 00:00:00     Informado   
19512  2024-02-01 00:00:00  2024-02-01 00:00:00     Informado   
19513  2024-12-27 00:00:00                  NaN  Por Informar   
19514  2024-05-13 00:00:00  2024-05-13 00:00:00     Informado   
19515  2024-05-13 00:00:00  2024-05-13 00:00:00     Informado   

      Monto Neto Documento Monto Impuestos Documento Monto Exento Documento  \
19511                20151                      3829                      0   
19512                10076                      1914                      0   
19513               179358                     34078                      0   
19514                12597                      2393                      0   
19515                 5874                      1116                      0   

      Monto Documento Monto Adeudado ABONO DE CLIENTES CHEQUE CREDITO  \
19511           23980            NaN               NaN    NaN     NaN   
19512           11990            NaN               NaN    NaN     NaN   
19513          213436            NaN            119923      0  213436   
19514           14990            NaN                 0      0       0   
19515            6990            NaN                 0      0       0   

      EFECTIVO FACTORING Mercado Pago NOTA CREDITO DEVOLUCION TARJETA CREDITO  \
19511      NaN       NaN          NaN                     NaN             NaN   
19512      NaN       NaN          NaN                     NaN             NaN   
19513        0         0            0                       0               0   
19514    14990         0            0                       0               0   
19515        0         0            0                       0               0   

      TARJETA DEBITO TRANSFERENCIA BANCARIA WEBPAY  \
19511            NaN                    NaN    NaN   
19512            NaN                    NaN    NaN   
19513              0                      0      0   
19514              0                      0      0   
19515              0                      0      0   

      Numero Comprobante Integrado Nº Comprobante Banco Número BancoValue  \
19511                          NaN            NaN   NaN    NaN        NaN   
19512                          NaN            NaN   NaN    NaN        NaN   
19513                          NaN            NaN   NaN    NaN        NaN   
19514                          NaN            NaN   NaN    NaN        NaN   
19515                          NaN            NaN   NaN    NaN        NaN   

      FACTURA COMPRA FACTORING.1                Emisor      Vendedor  \
19511            NaN         NaN  San Mateo Spa ONLINE  VENTA ONLINE   
19512          

Set on tables conditions for Marketplace

In [15]:


# Set on facturas when Sucursal is SUCURSAL MAYORISTA set Marketplace as vpm
final_table['Marketplace'] = np.where(
    (final_table['Sucursal'] == 'SUCURSAL MAYORISTA') & 
    (final_table['Marketplace'] == 'Sin datos') & (final_table['Tipo Documento'] != 'BOLETA ELECTRÓNICA T'),
    'vpm',
    final_table['Marketplace']
)

# Set on facturas when Vendedor is VENTA ONLINE set Marketplace as fcom
final_table['Marketplace'] = np.where(
    (final_table['Vendedor'] == 'VENTA ONLINE') & 
    (final_table['Marketplace'] == 'Sin datos'),
    'fcom',
    final_table['Marketplace']
)

# Set on facturas when Sucursal is SUCURSAL MAYORISTA and BOLETA ELECTRÓNICA T set Marketplace as T.NAVIDEÑA
final_table['Marketplace'] = np.where(
    (final_table['Sucursal'] == 'SUCURSAL MAYORISTA') & 
    (final_table['Marketplace'] == 'Sin datos') & (final_table['Tipo Documento'] == 'BOLETA ELECTRÓNICA T'),
    'T.NAVIDEÑA',
    final_table['Marketplace']
)

# Set on facturas when Tipo Documento is FACTURA DE EXPORTACIÓN ELECTRÓNICA set Marketplace as cvd
final_table['Marketplace'] = np.where(
    (final_table['Tipo Documento'] == 'FACTURA DE EXPORTACIÓN ELECTRÓNICA') & 
    (final_table['Marketplace'] == 'Sin datos'),
    'cvd',
    final_table['Marketplace']
)

final_table.head()

# Verify conditions
print(final_table.loc[final_table['Tipo Documento'] == 'FACTURA DE EXPORTACIÓN ELECTRÓNICA', 'Marketplace'].value_counts())
print(final_table.loc[final_table['Vendedor'] == 'VENTA ONLINE', 'Marketplace'].value_counts())
print(final_table.loc[final_table['Sucursal'] == 'SUCURSAL MAYORISTA', 'Marketplace'].value_counts())
print(final_table.loc[(final_table['Sucursal'] == 'SUCURSAL MAYORISTA') & (final_table['Tipo Documento'] == 'BOLETA ELECTRÓNICA T'), 'Marketplace'].value_counts())



# Last update of the dictionary
marketplace_dict.update(final_table.set_index('Nº Documento')['Marketplace'].to_dict())



Marketplace
cvd    7
Name: count, dtype: int64
Marketplace
fcom      9510
vtex      7290
paris     1091
ripley     544
vpm        137
Name: count, dtype: int64
Marketplace
T.NAVIDEÑA    585
vpm           321
Name: count, dtype: int64
Marketplace
T.NAVIDEÑA    585
Name: count, dtype: int64


#### Working with principal

First we need to combine dictionarys with principal

In [16]:
# Clean principal
principal= principal.iloc[:-1] #Eliminate the last row
principal['SKU'] = principal['SKU'].astype(int).astype(str)
principal['SKU'] = principal['SKU'].str.replace('"', '', regex=False)
principal['SKU'] = principal['SKU'].str.strip()
principal['Costo neto unitario']=principal['Costo neto unitario'].astype(int)
principal['Cantidad']=principal['Cantidad'].astype(int)
principal['Costo Total Neto']=principal['Costo Total Neto'].astype(int)
principal['Número Documento']=principal['Número Documento'].astype(int).astype(str)
principal['Detalle de Productos/Servicios Pack/Promo']=principal['Detalle de Productos/Servicios Pack/Promo'].fillna(0).astype(int)

# fill NAN values when is object with "Sin datos"
principal[principal.select_dtypes(include='object').columns] = principal.select_dtypes(include='object').fillna("Sin datos")

# Tipo Documento with lowercase
principal['Tipo Documento'] = principal['Tipo Documento'].str.lower()


principal.info()
principal.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35438 entries, 0 to 35437
Data columns (total 39 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Tipo Movimiento                            35438 non-null  object 
 1   Tipo Documento                             35438 non-null  object 
 2   Número Documento                           35438 non-null  object 
 3   Fecha de Emisión                           35438 non-null  object 
 4   Tracking number                            35438 non-null  object 
 5   Fecha y Hora Venta                         35438 non-null  object 
 6   Sucursal                                   35438 non-null  object 
 7   Vendedor                                   35438 non-null  object 
 8   Cliente Nombre                             35438 non-null  object 
 9   Cliente RUT                                35438 non-null  object 
 10  Cliente Email         

Tipo Movimiento        Tipo Documento Número Documento Fecha de Emisión  \
0           venta  boleta electrónica t            21923       01/01/2024   
1           venta  boleta electrónica t            21924       01/01/2024   
2           venta  boleta electrónica t            21925       01/01/2024   
3           venta  boleta electrónica t            21926       01/01/2024   
4           venta  boleta electrónica t            21927       01/01/2024   

            Tracking number   Fecha y Hora Venta     Sucursal      Vendedor  \
0  659208472266bd053b66938c  31/12/2023 21:33:10  Casa Matriz  VENTA ONLINE   
1  659213862266bd053e66957b  31/12/2023 22:21:10  Casa Matriz  VENTA ONLINE   
2  659239f57e379f0505d14b6e  01/01/2024 01:05:09  Casa Matriz  VENTA ONLINE   
3  659239f9cb05a8054ae57022  01/01/2024 01:05:13  Casa Matriz  VENTA ONLINE   
4  65923dd57e379f0505d14ba0  01/01/2024 01:21:41  Casa Matriz  VENTA ONLINE   

     Cliente Nombre Cliente RUT              Cliente Email Cliente Dirección  \
0   viviana salinas  13617340-5                  Sin datos         Sin datos   
1     Silvia Psijas   6693321-0                  Sin datos         Sin datos   
2  Priscilla Bustos  16753486-4                  Sin datos         Sin datos   
3  Priscilla Bustos  16753486-4                  Sin datos         Sin datos   
4  Carolina Montiel  15663772-6  montieliglesias@gmail.com           Caren     

              Cliente Comuna Cliente Ciudad        Lista de precio  \
0  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   
1  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   
2  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   
3  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   
4  Regi\u00f3n Metropolitana       Santiago  Lista de Precios Base   

    Tipo de entrega Moneda Tipo de Producto/Servicio             SKU  \
0  retiro en tienda    CLP                     Botin  12231407015007   
1  retiro en tienda    CLP                     Botin  12231407023007   
2  retiro en tienda    CLP                  Sandalia  22323605015006   
3  retiro en tienda    CLP                  Sandalia  22323605015006   
4  retiro en tienda    CLP                      Bota  12231303015008   

                 Producto/Servicio  Variante  Otros Atributos  Marca  \
0    Via Uno Botin 12231407 Marron  37 Negro              NaN    NaN   
1    Via Uno Botin 12231407 Marron        37              NaN    NaN   
2  Via Uno Sandalia 22323605 Negro        36              NaN    NaN   
3  Via Uno Sandalia 22323605 Negro        36              NaN    NaN   
4      Via Uno Bota 12231303 Negro        38              NaN    NaN   

   Detalle de Productos/Servicios Pack/Promo  Precio base  \
0                                          0      10076.0   
1                                          0      10076.0   
2                                          0      12597.0   
3                                          0      12597.0   
4                                          0      10916.0   

   Precio Neto Unitario  Precio Bruto Unitario  Cantidad  Subtotal Neto  \
0               10076.0                  11990         1          10076   
1               10076.0                  11990         1          10076   
2               12597.0                  14990         1          12597   
3               12597.0                  14990         1          12597   
4               10916.0                  12990         1          10916   

   Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
0                1914           11990      Sin datos               0   
1                1914           11990      Sin datos               0   
2                2393           14990      Sin datos               0   
3                2393           14990      Sin datos               0   
4                2074           12990      Sin datos               0   

   Descuento Bruto  % Descuento  Costo neto u

In [17]:
#Create column Producto 
principal['Producto']=principal['SKU'].str[:8]

In [18]:
# For principal put dictionary of cost
principal.loc[(principal['Costo neto unitario'] == 0) & principal['SKU'].notna(), 'Costo neto unitario'] = (
    principal['SKU'].map(cost_dict))
principal['Costo neto unitario']=principal['Costo neto unitario'].fillna(0)
principal['Costo neto unitario']=principal['Costo neto unitario'].astype(int)
principal.tail()

Tipo Movimiento               Tipo Documento Número Documento  \
35433      devolucion  nota de crédito electrónica             6482   
35434      devolucion  nota de crédito electrónica             6482   
35435      devolucion  nota de crédito electrónica             6482   
35436      devolucion  nota de crédito electrónica             6482   
35437      devolucion  nota de crédito electrónica             6482   

      Fecha de Emisión Tracking number   Fecha y Hora Venta  \
35433       31/12/2024       Sin datos  27/12/2024 19:24:03   
35434       31/12/2024       Sin datos  27/12/2024 19:24:03   
35435       31/12/2024       Sin datos  27/12/2024 19:24:03   
35436       31/12/2024       Sin datos  27/12/2024 19:24:03   
35437       31/12/2024       Sin datos  27/12/2024 19:24:03   

                 Sucursal     Vendedor                       Cliente Nombre  \
35433  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35434  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35435  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35436  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35437  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   

      Cliente RUT       Cliente Email Cliente Dirección Cliente Comuna  \
35433  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35434  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35435  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35436  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35437  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   

      Cliente Ciudad     Lista de precio   Tipo de entrega Moneda  \
35433      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35434      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35435      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35436      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35437      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   

      Tipo de Producto/Servicio             SKU  \
35433                  Sandalia  22541611022006   
35434                  Sandalia  22541611022007   
35435                  Sandalia  22541611022008   
35436                  Sandalia  22541611022009   
35437                  Sandalia  22541611022010   

                     Producto/Servicio Variante  Otros Atributos  Marca  \
35433  Via Uno Sandalia 22541611 Verde       36              NaN    NaN   
35434  Via Uno Sandalia 22541611 Verde       37              NaN    NaN   
35435  Via Uno Sandalia 22541611 Verde       38              NaN    NaN   
35436  Via Uno Sandalia 22541611 Verde       39              NaN    NaN   
35437  Via Uno Sandalia 22541611 Verde       40              NaN    NaN   

       Detalle de Productos/Servicios Pack/Promo  Precio base  \
35433                                          0      17495.0   
35434                                          0      17495.0   
35435                                          0      17495.0   
35436                                          0      17495.0   
35437                                          0      17495.0   

       Precio Neto Unitario  Precio Bruto Unitario  Cantidad  Subtotal Neto  \
35433              -17495.0                  20819        -2         -34990   
35434              -17495.0                  20819        -3         -52485   
35435              -17495.0                  20819        -3         -52485   
35436              -17495.0                  20819        -2         -34990   
35437              -17495.0                  20819        -1         -17495   

       Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
35433               -6648          -41638      Sin datos               0   
35434               -9973          -62458      Sin datos               0   
35435        

In [19]:
# For principal put dictionary of count
principal.loc[(principal['Cantidad'] == 0) & principal['SKU'].notna(), 'Cantidad'] = (
    principal['SKU'].map(count_dict))
principal['Cantidad']=principal['Cantidad'].fillna(0)
principal['Cantidad']=principal['Cantidad'].astype(int)
principal.tail()

Tipo Movimiento               Tipo Documento Número Documento  \
35433      devolucion  nota de crédito electrónica             6482   
35434      devolucion  nota de crédito electrónica             6482   
35435      devolucion  nota de crédito electrónica             6482   
35436      devolucion  nota de crédito electrónica             6482   
35437      devolucion  nota de crédito electrónica             6482   

      Fecha de Emisión Tracking number   Fecha y Hora Venta  \
35433       31/12/2024       Sin datos  27/12/2024 19:24:03   
35434       31/12/2024       Sin datos  27/12/2024 19:24:03   
35435       31/12/2024       Sin datos  27/12/2024 19:24:03   
35436       31/12/2024       Sin datos  27/12/2024 19:24:03   
35437       31/12/2024       Sin datos  27/12/2024 19:24:03   

                 Sucursal     Vendedor                       Cliente Nombre  \
35433  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35434  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35435  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35436  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35437  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   

      Cliente RUT       Cliente Email Cliente Dirección Cliente Comuna  \
35433  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35434  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35435  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35436  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35437  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   

      Cliente Ciudad     Lista de precio   Tipo de entrega Moneda  \
35433      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35434      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35435      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35436      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35437      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   

      Tipo de Producto/Servicio             SKU  \
35433                  Sandalia  22541611022006   
35434                  Sandalia  22541611022007   
35435                  Sandalia  22541611022008   
35436                  Sandalia  22541611022009   
35437                  Sandalia  22541611022010   

                     Producto/Servicio Variante  Otros Atributos  Marca  \
35433  Via Uno Sandalia 22541611 Verde       36              NaN    NaN   
35434  Via Uno Sandalia 22541611 Verde       37              NaN    NaN   
35435  Via Uno Sandalia 22541611 Verde       38              NaN    NaN   
35436  Via Uno Sandalia 22541611 Verde       39              NaN    NaN   
35437  Via Uno Sandalia 22541611 Verde       40              NaN    NaN   

       Detalle de Productos/Servicios Pack/Promo  Precio base  \
35433                                          0      17495.0   
35434                                          0      17495.0   
35435                                          0      17495.0   
35436                                          0      17495.0   
35437                                          0      17495.0   

       Precio Neto Unitario  Precio Bruto Unitario  Cantidad  Subtotal Neto  \
35433              -17495.0                  20819        -2         -34990   
35434              -17495.0                  20819        -3         -52485   
35435              -17495.0                  20819        -3         -52485   
35436              -17495.0                  20819        -2         -34990   
35437              -17495.0                  20819        -1         -17495   

       Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
35433               -6648          -41638      Sin datos               0   
35434               -9973          -62458      Sin datos               0   
35435        

# Preparing Data principal

In [20]:

# Quantities for nota de crédito electrónica are negative
principal= apply_negative_abs(principal, 'Tipo Documento', 'nota de crédito electrónica', ['Cantidad'])

# Obtain the total cost (for notas de crédito electrónica the cost and total cost are positive)
principal.loc[principal['Tipo Documento'] == 'nota de crédito electrónica', 'Costo Total Neto'] = abs(
    principal['Costo neto unitario'] * principal['Cantidad']
)

# Subtotal Neto, Subtotal Impuestos ,Precio base and Subtotal Bruto are positive for bolenta electrónica t
principal.loc[principal['Tipo Documento'] == 'boleta electrónica t', ['Subtotal Neto', 'Subtotal Impuestos', 'Precio base', 'Subtotal Bruto']] = principal.loc[principal['Tipo Documento'] == 'boleta electrónica t', ['Subtotal Neto', 'Subtotal Impuestos', 'Precio base', 'Subtotal Bruto']].abs()

# Costo Total Neto is negative for boleta electrónica t 
principal= apply_negative_abs(principal, 'Tipo Documento', 'boleta electrónica t', ['Costo Total Neto','Costo neto unitario'])

# Subtotal Neto, Subtotal Impuestos ,Precio base and Subtotal Bruto are negative for nota de crédito electrónica
principal= apply_negative_abs(principal, 'Tipo Documento', 'nota de crédito electrónica', ['Subtotal Neto', 'Subtotal Impuestos', 'Precio base', 'Subtotal Bruto'])

# Costo Total Neto and Costo neto unitario is positive for nota de crédito electrónica
principal.loc[principal['Tipo Documento'] == 'nota de crédito electrónica', ['Costo Total Neto','Costo neto unitario']] = principal.loc[principal['Tipo Documento'] == 'nota de crédito electrónica', ['Costo Total Neto','Costo neto unitario']].abs()

principal.tail()



Tipo Movimiento               Tipo Documento Número Documento  \
35433      devolucion  nota de crédito electrónica             6482   
35434      devolucion  nota de crédito electrónica             6482   
35435      devolucion  nota de crédito electrónica             6482   
35436      devolucion  nota de crédito electrónica             6482   
35437      devolucion  nota de crédito electrónica             6482   

      Fecha de Emisión Tracking number   Fecha y Hora Venta  \
35433       31/12/2024       Sin datos  27/12/2024 19:24:03   
35434       31/12/2024       Sin datos  27/12/2024 19:24:03   
35435       31/12/2024       Sin datos  27/12/2024 19:24:03   
35436       31/12/2024       Sin datos  27/12/2024 19:24:03   
35437       31/12/2024       Sin datos  27/12/2024 19:24:03   

                 Sucursal     Vendedor                       Cliente Nombre  \
35433  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35434  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35435  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35436  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35437  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   

      Cliente RUT       Cliente Email Cliente Dirección Cliente Comuna  \
35433  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35434  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35435  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35436  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   
35437  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      TALAGANTE   

      Cliente Ciudad     Lista de precio   Tipo de entrega Moneda  \
35433      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35434      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35435      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35436      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35437      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   

      Tipo de Producto/Servicio             SKU  \
35433                  Sandalia  22541611022006   
35434                  Sandalia  22541611022007   
35435                  Sandalia  22541611022008   
35436                  Sandalia  22541611022009   
35437                  Sandalia  22541611022010   

                     Producto/Servicio Variante  Otros Atributos  Marca  \
35433  Via Uno Sandalia 22541611 Verde       36              NaN    NaN   
35434  Via Uno Sandalia 22541611 Verde       37              NaN    NaN   
35435  Via Uno Sandalia 22541611 Verde       38              NaN    NaN   
35436  Via Uno Sandalia 22541611 Verde       39              NaN    NaN   
35437  Via Uno Sandalia 22541611 Verde       40              NaN    NaN   

       Detalle de Productos/Servicios Pack/Promo  Precio base  \
35433                                          0     -17495.0   
35434                                          0     -17495.0   
35435                                          0     -17495.0   
35436                                          0     -17495.0   
35437                                          0     -17495.0   

       Precio Neto Unitario  Precio Bruto Unitario  Cantidad  Subtotal Neto  \
35433              -17495.0                  20819        -2         -34990   
35434              -17495.0                  20819        -3         -52485   
35435              -17495.0                  20819        -3         -52485   
35436              -17495.0                  20819        -2         -34990   
35437              -17495.0                  20819        -1         -17495   

       Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
35433               -6648          -41638      Sin datos               0   
35434               -9973          -62458      Sin datos               0   
35435        

Int for Precio base, Precio Neto Unitario, Precio Bruto Unitario, Cantidad, Subtotal Neto, Subtotal Impuestos, Subtotal Bruto.

In [21]:
# Precio base	Precio Neto Unitario	Precio Bruto Unitario	Cantidad	Subtotal Neto	Subtotal Impuestos	Subtotal Bruto is int
principal['Precio base'] = principal['Precio base'].astype(int)
principal['Precio Neto Unitario'] = principal['Precio Neto Unitario'].astype(int)
principal['Precio Bruto Unitario'] = principal['Precio Bruto Unitario'].astype(int)
principal['Cantidad'] = principal['Cantidad'].astype(int)
principal['Subtotal Neto'] = principal['Subtotal Neto'].astype(int)
principal['Subtotal Impuestos'] = principal['Subtotal Impuestos'].astype(int)
principal['Subtotal Bruto'] = principal['Subtotal Bruto'].astype(int)
principal['Costo Total Neto'] = principal['Costo Total Neto'].astype(int)
principal['Costo neto unitario'] = principal['Costo neto unitario'].astype(int)


Correcting community client

In [22]:
# View Client Comuna unique values
print(principal['Cliente Comuna'].unique())


['Regi\\u00f3n Metropolitana' 'Sin datos' 'Puente Alto' 'Viña Del Mar'
 'Las Condes' 'STGO' 'Lo Barnechea' 'Santiago' 'santiago' 'Providencia'
 'Peñalolén' 'San Miguel' 'CAÑETE' 'SANTIAGO' 'Pudahuel' 'La Florida'
 'Vitacura' 'Recoleta' 'Colina' 'Quilicura' 'Rancagua' 'Maipú'
 'LA FLORIDA' 'antofagasta' 'Estación Central' 'la reina ' 'LAS CONDES'
 'Ñuñoa' 'RANCAGUA' 'Macul' 'Linares' 'Concón' 'Puerto Montt' 'Parral'
 'Antofagasta' 'COYHAIQUE' 'ARAUCO' 'LANCO' 'OSORNO' 'TRAIGUEN' 'TOME'
 'TALAGANTE' 'TALCAHUANO' 'QUILLOTA' 'La Cisterna' 'LOS ANDES'
 'PROVIDENCIA' 'La Reina' 'El Bosque' 'Hualpén' 'Los Ángeles' 'CALAMA'
 'Huechuraba' 'Lo Espejo' 'Concepción' 'LAMPA' 'San Pedro De la Paz'
 'Quinta Normal' 'Los Andes' 'PUENTE ALTO' 'EXTRANJERO' 'Copiapó'
 'CABRERO' 'SANTAIGO' 'San Felipe' 'Bulnes' 'PUERTO MONTT' 'Temuco'
 'FRUTILLAR' 'Renca' 'CONCON' 'Chiguayante' 'Talca' 'REGION METROPOLITANA'
 'xxx' 'huechuraba' 'Independencia' 'Iquique' 'INDEPENDENCIA' 'Huecguraba'
 'SAN MIGUEL']


In [23]:
# Correct Cliente Comuna
principal['Cliente Comuna']=principal['Cliente Comuna'].str.lower()
principal['Cliente Comuna']=principal['Cliente Comuna'].str.replace("stgo", "santiago", regex=False)
principal['Cliente Comuna']=principal['Cliente Comuna'].str.replace("regi\\u00f3n metropolitana", "region metropolitana", regex=False)
principal['Cliente Comuna']=principal['Cliente Comuna'].str.replace("santaigo", "santiago", regex=False)
principal['Cliente Comuna']=principal['Cliente Comuna'].str.replace("quinta noregion metropolitanaal", "region metropolitana", regex=False)
principal['Cliente Comuna']=principal['Cliente Comuna'].str.replace("huecguraba", "huechuraba", regex=False)

print(principal['Cliente Comuna'].unique())

['region metropolitana' 'sin datos' 'puente alto' 'viña del mar'
 'las condes' 'santiago' 'lo barnechea' 'providencia' 'peñalolén'
 'san miguel' 'cañete' 'pudahuel' 'la florida' 'vitacura' 'recoleta'
 'colina' 'quilicura' 'rancagua' 'maipú' 'antofagasta' 'estación central'
 'la reina ' 'ñuñoa' 'macul' 'linares' 'concón' 'puerto montt' 'parral'
 'coyhaique' 'arauco' 'lanco' 'osorno' 'traiguen' 'tome' 'talagante'
 'talcahuano' 'quillota' 'la cisterna' 'los andes' 'la reina' 'el bosque'
 'hualpén' 'los ángeles' 'calama' 'huechuraba' 'lo espejo' 'concepción'
 'lampa' 'san pedro de la paz' 'quinta normal' 'extranjero' 'copiapó'
 'cabrero' 'san felipe' 'bulnes' 'temuco' 'frutillar' 'renca' 'concon'
 'chiguayante' 'talca' 'xxx' 'independencia' 'iquique']


In [24]:
# In some cases in Tipo de Producto/Servicio Zapatilla is written as Zapatillas
principal['Tipo de Producto/Servicio']=principal['Tipo de Producto/Servicio'].str.replace("Zapatillas", "Zapatilla", regex=False)

In [25]:
# Modify with the correct Margen and % Margen for each row
principal['Margen'] = principal['Precio Neto Unitario'] * principal['Cantidad'] - abs(principal['Costo Total Neto'])
principal['% Margen'] = principal['Margen'] / (principal['Precio Neto Unitario']* abs(principal['Cantidad'])) *100
principal['% Margen'] = principal['% Margen'].round(2).fillna(0) #We want to show the percentage with 2 decimals
principal.head()

Tipo Movimiento        Tipo Documento Número Documento Fecha de Emisión  \
0           venta  boleta electrónica t            21923       01/01/2024   
1           venta  boleta electrónica t            21924       01/01/2024   
2           venta  boleta electrónica t            21925       01/01/2024   
3           venta  boleta electrónica t            21926       01/01/2024   
4           venta  boleta electrónica t            21927       01/01/2024   

            Tracking number   Fecha y Hora Venta     Sucursal      Vendedor  \
0  659208472266bd053b66938c  31/12/2023 21:33:10  Casa Matriz  VENTA ONLINE   
1  659213862266bd053e66957b  31/12/2023 22:21:10  Casa Matriz  VENTA ONLINE   
2  659239f57e379f0505d14b6e  01/01/2024 01:05:09  Casa Matriz  VENTA ONLINE   
3  659239f9cb05a8054ae57022  01/01/2024 01:05:13  Casa Matriz  VENTA ONLINE   
4  65923dd57e379f0505d14ba0  01/01/2024 01:21:41  Casa Matriz  VENTA ONLINE   

     Cliente Nombre Cliente RUT              Cliente Email Cliente Dirección  \
0   viviana salinas  13617340-5                  Sin datos         Sin datos   
1     Silvia Psijas   6693321-0                  Sin datos         Sin datos   
2  Priscilla Bustos  16753486-4                  Sin datos         Sin datos   
3  Priscilla Bustos  16753486-4                  Sin datos         Sin datos   
4  Carolina Montiel  15663772-6  montieliglesias@gmail.com           Caren     

         Cliente Comuna Cliente Ciudad        Lista de precio  \
0  region metropolitana       Santiago  Lista de Precios Base   
1  region metropolitana       Santiago  Lista de Precios Base   
2  region metropolitana       Santiago  Lista de Precios Base   
3  region metropolitana       Santiago  Lista de Precios Base   
4  region metropolitana       Santiago  Lista de Precios Base   

    Tipo de entrega Moneda Tipo de Producto/Servicio             SKU  \
0  retiro en tienda    CLP                     Botin  12231407015007   
1  retiro en tienda    CLP                     Botin  12231407023007   
2  retiro en tienda    CLP                  Sandalia  22323605015006   
3  retiro en tienda    CLP                  Sandalia  22323605015006   
4  retiro en tienda    CLP                      Bota  12231303015008   

                 Producto/Servicio  Variante  Otros Atributos  Marca  \
0    Via Uno Botin 12231407 Marron  37 Negro              NaN    NaN   
1    Via Uno Botin 12231407 Marron        37              NaN    NaN   
2  Via Uno Sandalia 22323605 Negro        36              NaN    NaN   
3  Via Uno Sandalia 22323605 Negro        36              NaN    NaN   
4      Via Uno Bota 12231303 Negro        38              NaN    NaN   

   Detalle de Productos/Servicios Pack/Promo  Precio base  \
0                                          0        10076   
1                                          0        10076   
2                                          0        12597   
3                                          0        12597   
4                                          0        10916   

   Precio Neto Unitario  Precio Bruto Unitario  Cantidad  Subtotal Neto  \
0                 10076                  11990         1          10076   
1                 10076                  11990         1          10076   
2                 12597                  14990         1          12597   
3                 12597                  14990         1          12597   
4                 10916                  12990         1          10916   

   Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
0                1914           11990      Sin datos               0   
1                1914           11990      Sin datos               0   
2                2393           14990      Sin datos               0   
3                2393           14990      Sin datos               0   
4                2074           12990      Sin datos               0   

   Descuento Bruto  % Descuento  Costo neto unitario  Costo Total Neto  \
0

We need to set the Marketplace on principal using the dictionary obtained earlier.

In [26]:
#set Marketplace for principal
principal['Marketplace'] = principal['Número Documento'].map(marketplace_dict).fillna("Sin datos")

# All rows on Marketplace with lowercase
principal['Marketplace'] = principal['Marketplace'].str.lower()

principal.tail()

Tipo Movimiento               Tipo Documento Número Documento  \
35433      devolucion  nota de crédito electrónica             6482   
35434      devolucion  nota de crédito electrónica             6482   
35435      devolucion  nota de crédito electrónica             6482   
35436      devolucion  nota de crédito electrónica             6482   
35437      devolucion  nota de crédito electrónica             6482   

      Fecha de Emisión Tracking number   Fecha y Hora Venta  \
35433       31/12/2024       Sin datos  27/12/2024 19:24:03   
35434       31/12/2024       Sin datos  27/12/2024 19:24:03   
35435       31/12/2024       Sin datos  27/12/2024 19:24:03   
35436       31/12/2024       Sin datos  27/12/2024 19:24:03   
35437       31/12/2024       Sin datos  27/12/2024 19:24:03   

                 Sucursal     Vendedor                       Cliente Nombre  \
35433  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35434  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35435  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35436  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35437  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   

      Cliente RUT       Cliente Email Cliente Dirección Cliente Comuna  \
35433  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   
35434  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   
35435  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   
35436  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   
35437  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   

      Cliente Ciudad     Lista de precio   Tipo de entrega Moneda  \
35433      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35434      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35435      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35436      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35437      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   

      Tipo de Producto/Servicio             SKU  \
35433                  Sandalia  22541611022006   
35434                  Sandalia  22541611022007   
35435                  Sandalia  22541611022008   
35436                  Sandalia  22541611022009   
35437                  Sandalia  22541611022010   

                     Producto/Servicio Variante  Otros Atributos  Marca  \
35433  Via Uno Sandalia 22541611 Verde       36              NaN    NaN   
35434  Via Uno Sandalia 22541611 Verde       37              NaN    NaN   
35435  Via Uno Sandalia 22541611 Verde       38              NaN    NaN   
35436  Via Uno Sandalia 22541611 Verde       39              NaN    NaN   
35437  Via Uno Sandalia 22541611 Verde       40              NaN    NaN   

       Detalle de Productos/Servicios Pack/Promo  Precio base  \
35433                                          0       -17495   
35434                                          0       -17495   
35435                                          0       -17495   
35436                                          0       -17495   
35437                                          0       -17495   

       Precio Neto Unitario  Precio Bruto Unitario  Cantidad  Subtotal Neto  \
35433                -17495                  20819        -2         -34990   
35434                -17495                  20819        -3         -52485   
35435                -17495                  20819        -3         -52485   
35436                -17495                  20819        -2         -34990   
35437                -17495                  20819        -1         -17495   

       Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
35433               -6648          -41638      Sin datos               0   
35434               -9973          -62458      Sin datos               0   
35435        

Now we need to create a column named Producto. This can be obtained by truncating the SKU to the first 8 digits, and convert Fecha Documento to datetime

In [27]:
#Convert Fecha Documento, y Fecha y Hora Venta  to datetime
principal['Fecha de Emisión'] = pd.to_datetime(principal['Fecha de Emisión'], errors='coerce')
principal['Fecha y Hora Venta'] = pd.to_datetime(principal['Fecha y Hora Venta'], errors='coerce')

principal.tail()

Tipo Movimiento               Tipo Documento Número Documento  \
35433      devolucion  nota de crédito electrónica             6482   
35434      devolucion  nota de crédito electrónica             6482   
35435      devolucion  nota de crédito electrónica             6482   
35436      devolucion  nota de crédito electrónica             6482   
35437      devolucion  nota de crédito electrónica             6482   

      Fecha de Emisión Tracking number  Fecha y Hora Venta  \
35433              NaT       Sin datos 2024-12-27 19:24:03   
35434              NaT       Sin datos 2024-12-27 19:24:03   
35435              NaT       Sin datos 2024-12-27 19:24:03   
35436              NaT       Sin datos 2024-12-27 19:24:03   
35437              NaT       Sin datos 2024-12-27 19:24:03   

                 Sucursal     Vendedor                       Cliente Nombre  \
35433  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35434  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35435  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35436  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   
35437  SUCURSAL MAYORISTA  Maria Mella  CARCUR HERMANOS Y COMPANIA LIMITADA   

      Cliente RUT       Cliente Email Cliente Dirección Cliente Comuna  \
35433  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   
35434  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   
35435  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   
35436  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   
35437  80161100-1  carcur99@gmail.com    JOSE LEYAN 748      talagante   

      Cliente Ciudad     Lista de precio   Tipo de entrega Moneda  \
35433      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35434      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35435      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35436      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   
35437      TALAGANTE  Mayorista Inv 2023  retiro en tienda    CLP   

      Tipo de Producto/Servicio             SKU  \
35433                  Sandalia  22541611022006   
35434                  Sandalia  22541611022007   
35435                  Sandalia  22541611022008   
35436                  Sandalia  22541611022009   
35437                  Sandalia  22541611022010   

                     Producto/Servicio Variante  Otros Atributos  Marca  \
35433  Via Uno Sandalia 22541611 Verde       36              NaN    NaN   
35434  Via Uno Sandalia 22541611 Verde       37              NaN    NaN   
35435  Via Uno Sandalia 22541611 Verde       38              NaN    NaN   
35436  Via Uno Sandalia 22541611 Verde       39              NaN    NaN   
35437  Via Uno Sandalia 22541611 Verde       40              NaN    NaN   

       Detalle de Productos/Servicios Pack/Promo  Precio base  \
35433                                          0       -17495   
35434                                          0       -17495   
35435                                          0       -17495   
35436                                          0       -17495   
35437                                          0       -17495   

       Precio Neto Unitario  Precio Bruto Unitario  Cantidad  Subtotal Neto  \
35433                -17495                  20819        -2         -34990   
35434                -17495                  20819        -3         -52485   
35435                -17495                  20819        -3         -52485   
35436                -17495                  20819        -2         -34990   
35437                -17495                  20819        -1         -17495   

       Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
35433               -6648          -41638      Sin datos               0   
35434               -9973          -62458      Sin datos               0   
35435              

# Group by Marketplace table 

In [28]:
# Filter by rows that are not Despacho and envio
principal_group = principal[principal['Variante'] != 'Despacho']
principal_group = principal_group[principal_group['Variante'] != 'envio']

# Create a new column with the Precio Neto per row
principal_group['Precio Neto'] = principal_group['Subtotal Bruto'] - principal_group['Subtotal Impuestos']

# Group by Marketplace, Sucursal, Producto
principal_Marketplace = principal_group.groupby(['Marketplace']).agg(
    {'Cantidad': 'sum', 'Precio Neto': 'sum','Costo Total Neto': 'sum'}).reset_index()

# Calculate the margin
principal_Marketplace['Margen'] = principal_Marketplace['Precio Neto'] + principal_Marketplace['Costo Total Neto'] #It is positive because the cost is already negative
principal_Marketplace['% Margen'] = (principal_Marketplace['Margen'] / abs(principal_Marketplace['Precio Neto'])) * 100
principal_Marketplace['% Margen'] = principal_Marketplace['% Margen'].round(2).fillna(0)

# Order by % Margen
principal_Marketplace = principal_Marketplace.sort_values(by='% Margen', ascending=False)
principal_Marketplace['Tipo'] = np.where(principal_Marketplace['Cantidad'] < 0, 'Devolucion', 'Venta')
principal_Marketplace

Marketplace  Cantidad  Precio Neto  Costo Total Neto     Margen  % Margen  \
7         vpm      4094    111248426          64604887  175853313    158.07   
0      bodega         3        75939                 0      75939    100.00   
1         cvd         7     90236189                 0   90236189    100.00   
5   sin datos      1931     44384951         -19166781   25218170     56.82   
3       paris       581     15169346          -7234611    7934735     52.31   
4      ripley       451     11555426          -5983975    5571451     48.22   
2        fcom      4464     92273272         -52324890   39948382     43.29   
6  t.navideña      1342     18284737         -10613224    7671513     41.96   
8        vtex     10022    174153207        -117648967   56504240     32.45   

    Tipo  
7  Venta  
0  Venta  
1  Venta  
5  Venta  
3  Venta  
4  Venta  
2  Venta  
6  Venta  
8  Venta

# Group by Tipo de Producto / Servicio table

In [29]:
# Group by Tipo de Producto / Servicio 

principal_Producto = principal_group.groupby(['Tipo de Producto/Servicio']).agg(
    {'Cantidad': 'sum', 'Precio Neto': 'sum','Costo Total Neto': 'sum'}).reset_index() 
principal_Producto=principal_Producto.drop(principal_Producto[principal_Producto['Tipo de Producto/Servicio']=='Sin Tipo'].index)
# Calculate the margin
principal_Producto['Margen'] = principal_Producto['Precio Neto'] + principal_Producto['Costo Total Neto'] #It is positive because the cost is already negative
principal_Producto['% Margen'] = (principal_Producto['Margen'] / abs(principal_Producto['Precio Neto'])) * 100
principal_Producto['% Margen'] = principal_Producto['% Margen'].round(2).fillna(0)

# Order by % Margen
principal_Producto = principal_Producto.sort_values(by='% Margen', ascending=False)
principal_Producto['Tipo'] = np.where(principal_Producto['Cantidad'] < 0, 'Devolucion', 'Venta')
principal_Producto

Tipo de Producto/Servicio  Cantidad  Precio Neto  Costo Total Neto  \
1                     Botin      6688    162738560          -9682126   
6                    Zapato      2046     35987484         -11493330   
0                      Bota      2770     81722100         -28331838   
3                  Sandalia     10490    145811121         -93733403   
5                 Zapatilla       287      6036024          -3905166   
2                   Playera       182       887963          -1193532   

      Margen  % Margen   Tipo  
1  153056434     94.05  Venta  
6   24494154     68.06  Venta  
0   53390262     65.33  Venta  
3   52077718     35.72  Venta  
5    2130858     35.30  Venta  
2    -305569    -34.41  Venta

# Filter Product with % Margen <=20%

In [30]:
Low_Margen_Marketplace = principal_Marketplace[principal_Marketplace['% Margen'] <= 20]
Low_Margen_Producto = principal_Producto[principal_Producto['% Margen'] <= 20]

display(Low_Margen_Marketplace)
display(Low_Margen_Producto)



Empty DataFrame
Columns: [Marketplace, Cantidad, Precio Neto, Costo Total Neto, Margen, % Margen, Tipo]
Index: []

Tipo de Producto/Servicio  Cantidad  Precio Neto  Costo Total Neto  Margen  \
2                   Playera       182       887963          -1193532 -305569   

   % Margen   Tipo  
2    -34.41  Venta

# Export to Excel

In [31]:
def set_column_formatting(writer, df, sheet_name):
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    header_format = workbook.add_format({'bold': True, 'bg_color': '#F9DA04', 'border': 1})
    # Recorre cada columna para establecer ancho y formatear el encabezado
    for i, col in enumerate(df.columns):
        max_len = max(df[col].fillna("").astype(str).map(len).max(), len(col)) + 2
        worksheet.set_column(i, i, max_len)
        worksheet.write(0, i, col, header_format)

# Export principal.xlsx con cada DataFrame en una hoja separada
with pd.ExcelWriter("principal.xlsx", engine="xlsxwriter") as writer:
    # Hoja para el DataFrame principal
    principal.to_excel(writer, sheet_name="Principal", index=False)
    set_column_formatting(writer, principal, "Principal")
    
    # Hoja para principal_Marketplace
    principal_Marketplace.to_excel(writer, sheet_name="Principal Marketplace", index=False)
    set_column_formatting(writer, principal_Marketplace, "Principal Marketplace")
    
    # Hoja para principal_Producto
    principal_Producto.to_excel(writer, sheet_name="Principal Producto", index=False)
    set_column_formatting(writer, principal_Producto, "Principal Producto")
    
    # Hoja para Low_Margen_Marketplace
    Low_Margen_Marketplace.to_excel(writer, sheet_name="Low Margen Marketplace", index=False)
    set_column_formatting(writer, Low_Margen_Marketplace, "Low Margen Marketplace")
    
    # Hoja para Low_Margen_Producto
    Low_Margen_Producto.to_excel(writer, sheet_name="Low Margen Producto", index=False)
    set_column_formatting(writer, Low_Margen_Producto, "Low Margen Producto")

# Export final_table.xlsx en una única hoja ("Final Table")
with pd.ExcelWriter("final_table.xlsx", engine="xlsxwriter") as writer:
    final_table.to_excel(writer, sheet_name="Final Table", index=False)
    set_column_formatting(writer, final_table, "Final Table")